In [ ]:
import re
from datetime import datetime
import unicodedata

# Map for converting Japanese eras to Gregorian years
era_map = {
    "令和": 2018, 
    "平成": 1988, 
    "昭和": 1925, 
    "大正": 1911,
    "明治": 1867
}

# Function to normalize the input date to half-width characters
def normalize_to_halfwidth(input_date):
    return unicodedata.normalize('NFKC', input_date)

# Function to compute the day of the week based on the date
def get_day_of_week(current_date, day_format):
    # Normalize input to half-width characters
    current_date = normalize_to_halfwidth(current_date)
    
    # Match Gregorian date formats (e.g., YYYY/MM/DD, YYYY-MM-DD, YYYY.MM.DD, or with kanji)
    match = re.search(r'(\d{4})[-/\.](\d{1,2})[-/\.](\d{1,2})|(\d{4})年(\d{1,2})月(\d{1,2})日', current_date)
    if match:
        year = int(match.group(1) or match.group(4))
        month = int(match.group(2) or match.group(5))
        day = int(match.group(3) or match.group(6))
    else:
        # Match Japanese era formats (e.g., 令和6年12月9日)
        match = re.search(r'([^\d]+)(\d+)年\s*(\d{1,2})月\s*(\d{1,2})日', current_date)
        if match:
            era = match.group(1)
            year = int(match.group(2))
            month = int(match.group(3))
            day = int(match.group(4))
            if era in era_map:
                # Convert 和暦 year to Gregorian year
                year = era_map[era] + year
            else:
                return "Invalid era"
        else:
            return "Invalid date format"
    
    # Get the day of the week
    date_obj = datetime(year, month, day)
    english_day_name = date_obj.strftime("%A")  # Full name of the day in English

    # Map English day names to Japanese
    japanese_days = {
        "Monday": "月曜日", "Tuesday": "火曜日", "Wednesday": "水曜日",
        "Thursday": "木曜日", "Friday": "金曜日", "Saturday": "土曜日", "Sunday": "日曜日"
    }
    japanese_short_days = {k: v[0] for k, v in japanese_days.items()}  

    # Format the day based on the day_format input
    if day_format == "Mon": 
        return english_day_name[:3]
    elif day_format == "Monday": 
        return english_day_name
    elif day_format == "(Mon)": 
        return f"({english_day_name[:3]})"
    elif day_format == "(Monday)":  
        return f"({english_day_name})"
    elif day_format == "月":  
        return japanese_short_days[english_day_name]
    elif day_format == "月曜日": 
        return japanese_days[english_day_name]
    elif day_format == "(月)←半角" or day_format == "（月）←全角":  
        return f"({japanese_short_days[english_day_name]})"
    elif day_format == "(月曜)←半角" or day_format == "（月曜）←全角":  
        return f"({japanese_short_days[english_day_name]}曜)"
    elif day_format == "(月曜日)←半角" or day_format == "（月曜日）←全角":  
        return f"({japanese_days[english_day_name]})"
    else:
        return "Invalid format"

# Get the day of the week in the desired format
day_of_week = get_day_of_week(current_date, day_format)
